In [1]:
import numpy as np
import matplotlib.pyplot as plt
import wave
import struct
import tkinter as tk
from tkinter import filedialog, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

# Variable global para guardar la ruta del archivo
file_name = None

# Función para subir el archivo de audio
def subir_audio():
    global file_name
    try:
        # Abrir el diálogo para seleccionar un archivo .wav
        file_name = filedialog.askopenfilename(
            title="Seleccionar archivo de audio .wav",
            filetypes=[("Archivos WAV", "*.wav")]
        )

        if not file_name:
            raise ValueError("No se seleccionó ningún archivo.")

        # Cargar el archivo de audio .wav
        with wave.open(file_name, 'rb') as wav_file:
            params = wav_file.getparams()
            num_channels = params.nchannels
            sample_width = params.sampwidth

            # Aseguramos que el archivo sea de 16 bits estéreo
            if num_channels != 2 or sample_width != 2:
                raise ValueError("El archivo debe ser estéreo y de 16 bits.")
        
        # Si todo está bien, mostramos un mensaje de éxito
        messagebox.showinfo("Éxito", "Archivo subido con éxito. El archivo cumple con los parámetros.")
    
    except ValueError as ve:
        messagebox.showerror("Error", str(ve))
    except Exception as e:
        messagebox.showerror("Error inesperado", str(e))

# Función para procesar el archivo de audio
def procesar_audio():
    try:
        if not file_name:
            raise ValueError("Primero debe subir un archivo.")

        # Obtener la cantidad de muestras a graficar
        num_muestras = int(entry_muestras.get())
        if num_muestras <= 0:
            raise ValueError("El número de muestras debe ser mayor a cero.")

        # Cargar el archivo de audio .wav
        with wave.open(file_name, 'rb') as wav_file:
            params = wav_file.getparams()
            num_channels = params.nchannels
            sample_width = params.sampwidth
            sample_rate = params.framerate
            num_samples = params.nframes

            # Leemos los frames del archivo
            samples = wav_file.readframes(num_samples)

            # Convertir datos binarios a tipos de datos nativos de Python
            audio_data = np.array(struct.unpack(f'<{num_samples * num_channels}h', samples))
            audio_data = audio_data.reshape(-1, num_channels)  # Separar canales

        # Recortar las muestras solicitadas
        muestras_recortadas = audio_data[:num_muestras]

        # Mostrar el Sample Rate y la Profundidad de bits en la interfaz
        bit_depth = sample_width * 8  # Convertir de bytes a bits
        label_sample_rate.config(text=f"Frecuencia de muestreo: {sample_rate} Hz")
        label_bit_depth.config(text=f"Profundidad de bits: {bit_depth} bits")

        # Limpiar el canvas anterior
        for widget in frame_grafico.winfo_children():
            widget.destroy()

        # Crear una nueva figura de Matplotlib para integrar en Tkinter
        fig = Figure(figsize=(10, 5), dpi=100)

        # Añadir subplots para los dos canales
        for channel in range(muestras_recortadas.shape[1]):
            ax = fig.add_subplot(2, 1, channel + 1)
            ax.stem(muestras_recortadas[:, channel])  # Gráfico de los datos
            ax.set_title(f'Canal {channel + 1}')
            ax.set_xlabel('Muestra')
            ax.set_ylabel('Amplitud')

        # Mostrar la figura en el canvas de Tkinter
        canvas = FigureCanvasTkAgg(fig, master=frame_grafico)
        canvas.draw()
        canvas.get_tk_widget().pack()

    except ValueError as ve:
        messagebox.showerror("Error", str(ve))
    except Exception as e:
        messagebox.showerror("Error inesperado", str(e))

# Función para salir de la aplicación
def salir():
    root.quit()
    root.destroy()

# Crear la ventana principal
root = tk.Tk()
root.title("Procesador de Audio .WAV")

# Crear una etiqueta para los requisitos del archivo
label_requisitos = tk.Label(root, text="El archivo debe ser .wav, 16 bits, y estéreo", fg="red", font=("Helvetica", 12, "bold"))
label_requisitos.pack(pady=10)

# Botón para subir el archivo de audio
button_subir = tk.Button(root, text="Subir Archivo", command=subir_audio)
button_subir.pack(pady=10)

# Crear etiquetas y entradas para las muestras
label_muestras = tk.Label(root, text="Número de muestras a graficar:")
label_muestras.pack(pady=10)

entry_muestras = tk.Entry(root)
entry_muestras.pack(pady=10)

# Botón para procesar el archivo de audio
button_procesar = tk.Button(root, text="Procesar Archivo", command=procesar_audio)
button_procesar.pack(pady=20)

# Crear etiquetas para mostrar el Sample Rate y la Profundidad de bits
label_sample_rate = tk.Label(root, text="Frecuencia de muestreo: -- Hz")
label_sample_rate.pack(pady=5)

label_bit_depth = tk.Label(root, text="Profundidad de bits: -- bits")
label_bit_depth.pack(pady=5)

# Crear un frame para contener la gráfica
frame_grafico = tk.Frame(root)
frame_grafico.pack(pady=20, fill=tk.BOTH, expand=True)

# Botón para salir de la aplicación
button_salir = tk.Button(root, text="Salir", command=salir)
button_salir.pack(pady=20)

# Iniciar la interfaz gráfica
root.mainloop()